In [2]:
# 模型相关
import torch
from models.DIP_2D import DIP_2D
import pytorch_lightning as pl
import numpy as np

# 画图相关
import matplotlib.pyplot as plt

# 文件读写相关
import csv
from config import *
import os
import pandas as pd
import csv

# 自定义函数
from utils.pre_utils import *

import datetime
print(config['sub_iter_DIP'])

10000


In [3]:
# 第一个test，用于还原DIP paper中的那个用噪音文件学习不同图片的那个学习曲线learning_curve
# 输入是高斯噪音，目标分别是 ground_truth，含噪音图片，纯噪音(应该用另一个噪音的干！)
test_list = ["images/noise_images/ground_truth.npy","images/noise_images/BSREM_it30.npy","images/noise_images/uniform_noise2.npy"]


In [4]:

# input 这里我觉得可以测试不同的先测试一下高斯噪声吧，我觉得不同的噪声有可能对于学习曲线有影响,读取并标准化input
image_net_input = np.load("images/noise_images/uniform_noise.npy")
image_net_input_scale,param1_scale_im_net,param2_scale_im_net = rescale_imag(image_net_input,"standardization") 
image_net_input_torch = torch.Tensor(image_net_input_scale)
image_net_input_torch = image_net_input_torch.view(1,1,PETImage_shape[0],PETImage_shape[1],PETImage_shape[2])
image_net_input_torch = image_net_input_torch[:,:,:,:,0]


staaaaaaaaaaand


In [5]:

# 读取三个含噪声文件，将其做rescale和格式转换tensor 1,1,112,112
ground_truth = np.load("/home/xzhang/Documents/我的模型/images/noise_images/ground_truth.npy")

for path_noisy in test_list:
    print(path_noisy)
    image_corrupt=np.load(path_noisy) # 读取图片并将图片转换成numpy array
    image_corrupt_input_scaled,param1_scale_im_corrupt,param2_scale_im_corrupt = rescale_imag(image_corrupt,"standardization") # 标准化图片, 减去平均值，除以标准差，参数1是mean，参数2是std
    image_corrupt_torch = torch.Tensor(image_corrupt_input_scaled)
    image_corrupt_torch = image_corrupt_torch.view(1,1,PETImage_shape[0],PETImage_shape[1],PETImage_shape[2])
    image_corrupt_torch = image_corrupt_torch[:,:,:,:,0]

    # 用dataset和dataloader 读取训练数据 input和目标
    train_dataset = torch.utils.data.TensorDataset(image_net_input_torch, image_corrupt_torch)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1) 

    # 读取并导入模型
    model = DIP_2D(param1_scale_im_corrupt, param2_scale_im_corrupt,config,'data/Algo/',
                "nested",all_images_DIP="Last",global_it=-100, suffix="suffix", last_iter=-1,ground_truth=ground_truth)
    model_class = DIP_2D

    # 设置log路径保存在runs目录
    checkpoint_simple_path = 'test_logs'
    name=str(datetime.datetime.now())

    # 创建TensorBoardLogger，注意savedir和后面打开tensorboard时的路径要一致，versions是啥？
    logger = pl.loggers.TensorBoardLogger(save_dir=checkpoint_simple_path,name=name)
    print(config["sub_iter_DIP"])
    trainer = pl.Trainer(max_epochs=config["sub_iter_DIP"],log_every_n_steps=100,logger=logger)#, callbacks=[checkpoint_callback, tuning_callback, early_stopping_callback], logger=logger,gpus=gpus, accelerator=accelerator, profiler="simple")
    trainer.fit(model, train_dataloader)
    out = model(image_net_input_torch)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/xzhang/miniconda3/envs/dip/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1292: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(

   | Name       | Type       | Params
-------------------------------------------
0  | deep1      | Sequential | 2.5 K 
1  | down1      | Sequential | 2.4 K 
2  | deep2      | Sequential | 14.0 K
3  | down2      | Sequential | 9.3 K 
4  | deep3      | Sequential | 55.7 K
5  | down3      | Sequential | 37.1 K
6  | deep4      | Sequential | 221 K 
7  | up1        | Sequential | 73.9 K
8  | deep5      | Sequential | 74.1 K
9  | up2        | Sequential | 18.5 K
10 | deep6      | Sequential | 18.6 K
11 | up3        | Sequential | 4.7 K 
12 | deep7      | Sequential | 2.5 K 
13 | positivity | ReLU       | 0     
--------------------------------------

images/noise_images/ground_truth.npy
staaaaaaaaaaand
10000
Epoch 9999: 100%|██████████| 1/1 [00:00<00:00, 20.95it/s, loss=0.000157, v_num=0]   


GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

   | Name       | Type       | Params
-------------------------------------------
0  | deep1      | Sequential | 2.5 K 
1  | down1      | Sequential | 2.4 K 
2  | deep2      | Sequential | 14.0 K
3  | down2      | Sequential | 9.3 K 
4  | deep3      | Sequential | 55.7 K
5  | down3      | Sequential | 37.1 K
6  | deep4      | Sequential | 221 K 
7  | up1        | Sequential | 73.9 K
8  | deep5      | Sequential | 74.1 K
9  | up2        | Sequential | 18.5 K
10 | deep6      | Sequential | 18.6 K
11 | up3        | Sequential | 4.7 K 
12 | deep7      | Sequential | 2.5 K 
13 | positivity | ReLU       | 0     
-------------------------------------------
535 K     Trainable params
0         Non-trainable params
535 K     Total params
2.141     Total estimated model params size (MB)


images/noise_images/BSREM_it30.npy
staaaaaaaaaaand
10000
Epoch 9999: 100%|██████████| 1/1 [00:00<00:00, 19.40it/s, loss=0.00118, v_num=0]    

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

   | Name       | Type       | Params
-------------------------------------------
0  | deep1      | Sequential | 2.5 K 
1  | down1      | Sequential | 2.4 K 
2  | deep2      | Sequential | 14.0 K
3  | down2      | Sequential | 9.3 K 
4  | deep3      | Sequential | 55.7 K
5  | down3      | Sequential | 37.1 K
6  | deep4      | Sequential | 221 K 
7  | up1        | Sequential | 73.9 K
8  | deep5      | Sequential | 74.1 K
9  | up2        | Sequential | 18.5 K
10 | deep6      | Sequential | 18.6 K
11 | up3        | Sequential | 4.7 K 
12 | deep7      | Sequential | 2.5 K 
13 | positivity | ReLU       | 0     
-------------------------------------------
535 K     Trainable params
0         Non-trainable params
535 K     Total params
2.141     Total estimated model params size (MB)



images/noise_images/uniform_noise2.npy
staaaaaaaaaaand
10000
Epoch 9999: 100%|██████████| 1/1 [00:00<00:00, 20.66it/s, loss=0.0038, v_num=0]     
